This notebook recalculates ARBOC issuance based on values transcribed from project ODPRs.


In [ ]:
import fsspec
import os
import json

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from carbonplan_forest_offsets.analysis.allocation import get_arbocs
from carbonplan_forest_offsets.data import cat

from carbonplan import styles

styles.mpl.set_theme(style="carbonplan_light", font_scale=1.5)
colors = styles.colors.light

In [ ]:
project_db = cat.project_db_json.read()
arbocs = pd.DataFrame(
    {
        project["opr_id"]: (
            get_arbocs(project["baseline"], project["rp_1"]),
            project["arbocs"]["issuance"],
            project["arbocs"]["calculated"],
        )
        for project in project_db
    }
).T.rename(columns={0: "calculated", 1: "issued", 2: "old_calc"})

In [ ]:
d = (
    arbocs.reset_index()
    .rename(columns={"index": "opr_id"})[["opr_id", "calculated", "issued"]]
    .to_dict(orient="records")
)

In [ ]:
g = sns.FacetGrid(data=arbocs / 1_000_000, height=3.5)
g.map(plt.scatter, "calculated", "issued", s=125, color=".3", alpha=0.65)
g.axes[0][0].plot((0, 15), (0, 15), lw=3, ls="--", color="r", zorder=-9)
plt.xlabel("Calculated ARBOCs\n(Millions)")
plt.ylabel("Issued ARBOCs\n(Millions)")
plt.xlim(0, 15.5)
plt.ylim(0, 15.5)

g.axes[0][0].set_xticks(np.arange(0, 15.1, 5))
g.axes[0][0].set_yticks(np.arange(0, 15.1, 5))
# plt.savefig("/home/jovyan/pub-figs/recalculate_arbocs.pdf", dpi=300, bbox_inches="tight")

In [ ]:
with fsspec.open(
    "az://carbonplan-retro/results/crediting-verification.json",
    account_key=os.environ["BLOB_ACCOUNT_KEY"],
    account_name="carbonplan",
    mode="w",
) as f:
    json.dump(d, f)